<a href="https://colab.research.google.com/github/davidfague/Neural-Modeling/blob/main/Function_Group_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Full Simulation Run with new modules

#### Install Dependencies

In [1]:
!pip install neuron

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 57.5 MB/s eta 0:00:00


In [2]:
!pip install neuron_reduce

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Clone repo

In [3]:
!git clone https://github.com/davidfague/Neural-Modeling.git

Cloning into 'Neural-Modeling'...
remote: Enumerating objects: 273, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 273 (delta 105), reused 126 (delta 78), pack-reused 109
Receiving objects: 100% (273/273), 180.83 KiB | 6.46 MiB/s, done.
Resolving deltas: 100% (142/142), done.


Get modules

In [4]:
%cd Neural-Modeling/
from Modules.synapse_generator import SynapseGenerator
from Modules.Reduction import Reductor
from Modules.cell_model import CellModel
from Modules.spike_generator import SpikeGenerator

/content/Neural-Modeling


In [5]:
# others
# import pandas as pd
import numpy as np
from functools import partial
import scipy.stats as st
import random

#### Build Complex Cell

Compile modfiles

In [6]:
!nrnivmodl modfiles

/content/Neural-Modeling
Mod files: "modfiles/modfiles/AMPA_NMDA.mod" "modfiles/modfiles/AMPA_NMDA_STP_LTP.mod" "modfiles/modfiles/AMPA_NMDA_STP.mod" "modfiles/modfiles/CaDynamics_E2.mod" "modfiles/modfiles/Ca_HVA.mod" "modfiles/modfiles/Ca_LVAst.mod" "modfiles/modfiles/epsp.mod" "modfiles/modfiles/GABA_AB.mod" "modfiles/modfiles/GABA_AB_STP.mod" "modfiles/modfiles/Ih.mod" "modfiles/modfiles/Im.mod" "modfiles/modfiles/int2pyr.mod" "modfiles/modfiles/K_Pst.mod" "modfiles/modfiles/K_Tst.mod" "modfiles/modfiles/Nap_Et2.mod" "modfiles/modfiles/NaTa_t.mod" "modfiles/modfiles/NaTs2_t.mod" "modfiles/modfiles/pyr2pyr.mod" "modfiles/modfiles/SK_E2.mod" "modfiles/modfiles/SKv3_1.mod" "modfiles/modfiles/vecevent.mod"

Creating 'x86_64' directory for .o files.

 -> Compiling mod_func.cpp
 -> NMODL ../modfiles/AMPA_NMDA.mod
 -> NMODL ../modfiles/AMPA_NMDA_STP_LTP.mod
 -> NMODL ../modfiles/AMPA_NMDA_STP.mod
Translating AMPA_NMDA_STP_LTP.mod into /content/Neural-Modeling/x86_64/AMPA_NMDA_STP_LTP.c
Tr

In [7]:
from neuron import h
h.load_file('stdrun.hoc')
h.nrn_load_dll('./x86_64/.libs/libnrnmech.so') # load modfiles

1.0

Build Complex cell

In [8]:
cell_folder = 'complex_cells/L5PC/'
h.load_file(cell_folder+'L5PCbiophys3.hoc') # load biophysics
h.load_file("import3d.hoc") # load morphology
h.load_file(cell_folder+'L5PCtemplate.hoc') # load builder
complex_cell = h.L5PCtemplate(cell_folder+'cell1.asc') # build complex_cell object
h.celsius = 37
h.v_init = complex_cell.soma[0].e_pas
h.tstop = 2000 # sim runtime
t = np.arange(0,h.tstop-1,1) # time vector for generating inputs

In [9]:
fr_profile = np.zeros((t.shape[0]))
print(fr_profile)

[0. 0. 0. ... 0. 0. 0.]


Optional:
optomize complex cell nseg

In [10]:
Reductor(complex_cell, 'lambda')

Model nseg changed from 642 to 196


#### Generate excitatory synapses and their inputs

In [11]:
class FunctionalGroup():
  def __init__(self, cell, center_seg, span):
    self.center_seg=center_seg
    self.segments=[]
    self.len_per_segment=[]
    self.synapses=[]
    self.clusters=[]
    for sec in cell.all:
      for seg in sec:
        if h.distance(center_seg,seg)<span:
          self.segments.append(seg)
          self.len_per_segment.append(seg.sec.nseg/seg.sec.L)
    self.len_per_segment=np.array(self.len_per_segment)

class Cluster():
  def __init__(self, cell, center_seg, span, functional_group=None):
    self.center_seg=center_seg
    self.segments=[]
    self.len_per_segment=[]
    self.synapses=[]
    self.clusters=[]
    self.spike_trains=[]
    self.netcons_list=[]
    self.functional_group=functional_group
    if functional_group is not None:
      self.functional_group.clusters.append(self)
    for sec in cell.all:
      for seg in sec:
        if h.distance(center_seg,seg)<span:
          self.segments.append(seg)
          self.len_per_segment.append(seg.sec.nseg/seg.sec.L)
    self.len_per_segment=np.array(self.len_per_segment)

# get all segments
all_segments=[]
all_len_per_segment=[]
for sec in complex_cell.all:
  for seg in sec:
    all_segments.append(seg)
    all_len_per_segment.append(seg.sec.nseg/seg.sec.L)

all_len_per_segment=np.array(all_len_per_segment)
m=.2
s=0.345
gmax_mean = np.log(m) - 0.5 * np.log((s/m)**2+1)
gmax_std = np.sqrt(np.log((s/m)**2 + 1))
gmax_exc_dist = partial(np.random.lognormal, gmax_mean, gmax_std, size=1) # gmax distribution

# create functional groups
num_groups=52
functional_group_span=100
functional_groups=[]
nodes_per_group=100 # numbet of presynaptic cells
cluster_span=10
synapses_per_node=5
mean_fr_dist=partial(st.levy_stable.rvs, alpha=1.37, beta=-1.00, loc=0.92, scale=0.44, size=1) # distribution of mean firing rates
spike_generator=SpikeGenerator()
synapse_generator=SynapseGenerator()
t=np.arange(0,h.tstop,1)

rnd = np.random.RandomState(10)
for cluster in range(num_groups): # create functional group
  center_seg=rnd.choice(all_segments, p=all_len_per_segment/sum(all_len_per_segment))
  func_grp=FunctionalGroup(complex_cell, center_seg, functional_group_span)
  functional_groups.append(func_grp)
  fr_profile=spike_generator.get_firing_rate_profile(t, mean_firing_rate=mean_fr_dist, method='1f_noise')
  for cell in range(nodes_per_group):
    cluster_seg=rnd.choice(func_grp.segments, p=func_grp.len_per_segment/sum(func_grp.len_per_segment))
    cluster=Cluster(complex_cell, cluster_seg, cluster_span)
    cluster.synapses=synapse_generator.add_synapses(segments=cluster.segments,probs=cluster.len_per_segment,gmax=gmax_exc_dist,syn_mod='AMPA_NMDA',number_of_synapses=synapses_per_node)
    func_grp.synapses.append(cluster.synapses)
    func_grp.clusters.append(cluster)
    mean_fr=spike_generator.get_mean_fr(mean_fr_dist)
    spikes = spike_generator.generate_spikes_from_profile(fr_profile,mean_fr)
    for synapse in cluster.synapses:
      print(synapse, synapse.pp_obj)
      cluster.netcons_list.append(spike_generator.set_spike_train(synapse, spikes))
      cluster.spike_trains.append(spikes)




<Modules.synapse.Synapse object at 0x7f2e4b7e8ca0> AMPA_NMDA[0]
<Modules.synapse.Synapse object at 0x7f2e4b7e9030> AMPA_NMDA[1]
<Modules.synapse.Synapse object at 0x7f2e4b7e8fd0> AMPA_NMDA[2]
<Modules.synapse.Synapse object at 0x7f2e4b7e9000> AMPA_NMDA[3]
<Modules.synapse.Synapse object at 0x7f2e4b7e9fc0> AMPA_NMDA[4]
<Modules.synapse.Synapse object at 0x7f2e4b7ebf40> AMPA_NMDA[5]
<Modules.synapse.Synapse object at 0x7f2e4b7ebf10> AMPA_NMDA[6]
<Modules.synapse.Synapse object at 0x7f2e4b7ebee0> AMPA_NMDA[7]
<Modules.synapse.Synapse object at 0x7f2e4b7ebeb0> AMPA_NMDA[8]
<Modules.synapse.Synapse object at 0x7f2e4b7ebe80> AMPA_NMDA[9]
<Modules.synapse.Synapse object at 0x7f2e4b7eb8b0> AMPA_NMDA[10]
<Modules.synapse.Synapse object at 0x7f2e4b7eb880> AMPA_NMDA[11]
<Modules.synapse.Synapse object at 0x7f2e4b7eb850> AMPA_NMDA[12]
<Modules.synapse.Synapse object at 0x7f2e4b7eb820> AMPA_NMDA[13]
<Modules.synapse.Synapse object at 0x7f2e4b7eb7f0> AMPA_NMDA[14]


ValueError: ignored

In [ ]:
for _ in fr_profile:
  print(_)

In [ ]:
for synapse in cluster.synapses:
  print(synapse.pp_obj)

In [ ]:
SynapseGenerator_exc = SynapseGenerator() # will generate and store new synapses lists

Excitatory synapses

In [ ]:
# # add intracortical and cortico-cortical pyramidal synapses to basal dendrites
# segments=[] # list of nrn.segment objects
# for sec in complex_cell.dend: # basal only
#   for seg in sec:
#     segments.append(seg)

# len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in segments]) # list of random choice probabilities

# m=.2
# s=0.345
# gmax_mean = np.log(m) - 0.5 * np.log((s/m)**2+1)
# gmax_std = np.sqrt(np.log((s/m)**2 + 1))
# gmax_exc_dist = partial(np.random.lognormal, gmax_mean, gmax_std, size=1) # gmax distribution

# intracortical_pyr = SynapseGenerator_exc.add_synapses(segments, gmax=gmax_exc_dist, syn_mod='AMPA_NMDA', density=2.16, probs = len_per_segment, record = False)

In [ ]:
# possibly split synapse list between intracortical and cortico-cortical # cortico-cortical may tend to have stronger weight

In [ ]:
# # add thalamocortical and cortico-cortical pyramidal synapses to distal apical dendrites
# segments=[] # list of nrn.segment objects
# for sec in complex_cell.apic: # distal apical only
#   if h.distance(complex_cell.soma[0](0.5),sec(0.5))<700:
#     for seg in sec:
#       segments.append(seg)

# len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in segments]) # list of random choice probabilities

# m=.2
# s=0.345
# gmax_mean = np.log(m) - 0.5 * np.log((s/m)**2+1)
# gmax_std = np.sqrt(np.log((s/m)**2 + 1))
# gmax_exc_dist = partial(np.random.lognormal, gmax_mean, gmax_std, size=1) # gmax distribution

# thalamocortical_pyr = SynapseGenerator_exc.add_synapses(segments=segments, gmax=gmax_exc_dist, syn_mod='AMPA_NMDA', density=2.16, probs = len_per_segment, record = False)

In [ ]:
# possible split synapse list between thalamocortical and cortico-cortical

Excitatory synaptic Inputs

In [ ]:
# SpikeGenerator_exc = SpikeGenerator() # will generate and store new netcons

In [ ]:
# levy_dist = partial(st.levy_stable.rvs, alpha=1.37, beta=-1.00, loc=0.92, scale=0.44, size=1) # distribution of mean firing rates

# # intracortical and cortico-cortical pyramidal synapses on basal dendrites
# netcons_list, spike_trains = SpikeGenerator_exc.generate_inputs(synapses=intracortical_pyr, t=t, method = '1/f noise', mean_firing_rate=levy_dist) # possibly use same_presynaptic_cell/same_presynaptic_region after splitting lists
# # thalamocortical and cortico-cortical pyramidal synapses on distal apical dendrites
# netcons_list, spike_trains = SpikeGenerator_exc.generate_inputs(synapses=thalamocortical_pyr, t=t, method = '1/f noise', mean_firing_rate=levy_dist) # possibly use same_presynaptic_cell/same_presynaptic_region after splitting lists)

#### Generate inhibitory synapses and their inputs

Inhibitory synapses

In [ ]:
SynapseGenerator_inh = SynapseGenerator() # will generate and store new synapses lists

In [ ]:
# add proximal intracortical interneuron synapses everywhere at lower density
segments=[] # list of nrn.segment objects
for sec in complex_cell.all:
  if h.distance(complex_cell.soma[0](0.5),sec(0.5))<50: # proximal
    for seg in sec:
      segments.append(seg)

len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in segments]) # list of random choice probabilities

m=.1
s=0.345
gmax_mean = np.log(m) - 0.5 * np.log((s/m)**2+1)
gmax_std = np.sqrt(np.log((s/m)**2 + 1))
gmax_inh_dist = partial(np.random.lognormal, gmax_mean, gmax_std, size=1) # gmax distribution

prox_intracortical_int = SynapseGenerator_inh.add_synapses(segments, gmax=gmax_inh_dist, syn_mod='GABA_AB', density=0.22, probs = len_per_segment, record = False)

In [ ]:
# add distal intracortical interneuron synapses everywhere at lower density
segments=[] # list of nrn.segment objects
for sec in complex_cell.all:
  if h.distance(complex_cell.soma[0](0.5),sec(0.5))>50: # distal
    for seg in sec:
      segments.append(seg)

len_per_segment = np.array([seg.sec.L/seg.sec.nseg for seg in segments]) # list of random choice probabilities

m=.1
s=0.345
gmax_mean = np.log(m) - 0.5 * np.log((s/m)**2+1)
gmax_std = np.sqrt(np.log((s/m)**2 + 1))
gmax_inh_dist = partial(np.random.lognormal, gmax_mean, gmax_std, size=1) # gmax distribution

dist_intracortical_int = SynapseGenerator_inh.add_synapses(segments, gmax=gmax_inh_dist, syn_mod='GABA_AB', density=0.22, probs = len_per_segment, record = False)

Inhibitroy synaptic Inputs

In [ ]:
SpikeGenerator_inh = SpikeGenerator() # will generate and store new netcons

In [ ]:
# proximal inh mean_fr distribution
mean_fr, std_fr = 16.9, 14.3
a, b = (0 - mean_fr) / std_fr, (100 - mean_fr) / std_fr
proximal_inh_dist = partial(st.truncnorm.rvs, a=a, b=b, loc=mean_fr, scale=std_fr)

# intracortical and cortico-cortical pyramidal synapses on basal dendrites
netcons_list, spike_trains = SpikeGenerator_inh.generate_inputs(synapses=prox_intracortical_int, t=t, method = 'delay', mean_firing_rate=proximal_inh_dist,) # possibly use same_presynaptic_cell/same_presynaptic_region after splitting lists

In [ ]:
# distal inh mean_fr distribution
mean_fr, std_fr = 3.9, 4.3
a, b = (0 - mean_fr) / std_fr, (100 - mean_fr) / std_fr
distal_inh_dist = partial(st.truncnorm.rvs, a=a, b=b, loc=mean_fr, scale=std_fr)

